In [36]:
from selenium import webdriver
from bs4 import BeautifulSoup 
import pandas as pd
import csv

# Open web automaticly
driver=webdriver.Edge()
url='https://www.thegioididong.com' #Lấy link để mở web tự động
driver.get(url)
dt=driver.find_element_by_xpath('/html/body/header/div[2]/div/ul/li[1]/a/span') 
dt.click()


#Scrape data of products from soucre web 
def GetUrl():
    page_s=BeautifulSoup(driver.page_source) #dùng bs4 tải toàn bộ HTML document lên chương trình
    tel_infors=page_s.find_all('a',class_='main-contain') # tìm đúng element trong source để lấy chính xác id, ở đây sử dụng class và  task a

    all_prod=[]
    for tel_infor in tel_infors:
        tel_ID=tel_infor.get('href') # lấy link đuôi trong thành phần có tên là href
        tel_If_url='https://www.thegioididong.com'+tel_ID
        if tel_If_url not in all_prod:  #để tránh lặp lại gá trị vì trong web có các thành phần nằm chồng lên nhau và giống nhau
            all_prod.append(tel_If_url)
        set(all_prod)
    return all_prod
#print(GetUrl(),'\n==============================================================================')



C:\Users\Admin\AppData\Local\Temp\ipykernel_19704\2362635356.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  dt=driver.find_element_by_xpath('/html/body/header/div[2]/div/ul/li[1]/a/span')


Lặp từng vòng page để lấy dữ liệu

In [37]:
#numPg=int(input('Input the page number do you want to scrape'))
numPg=4
    
Url_all=[]
  
for i in range(numPg):
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight)') # thực hiện kéo xuống xuối cuối trang 
    next_btn=driver.find_element_by_xpath('//*[@id="categoryPage"]/div[3]/div[2]/a')
    next_btn.click()
driver.execute_script('window.scrollTo(0,document.body.scrollHeight)') # thực hiện kéo xuống xuối cuối trang 
Url_each_page=GetUrl() # Lấy thông tin tất cả từng trang
Url_all=Url_all+Url_each_page  




C:\Users\Admin\AppData\Local\Temp\ipykernel_19704\161496726.py:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  next_btn=driver.find_element_by_xpath('//*[@id="categoryPage"]/div[3]/div[2]/a')


Write into file .csv

In [38]:

with open('Du_lieu_dien_thoai2.csv','w',newline='',encoding='utf-8')as file_output:
    header=['Name','Display','HDH','Camera sau','Camera trước','Chip','Ram','Rom','Sim','Battery','Charge','Gia','Danh gia','So Luong']
    writer =csv.DictWriter(file_output,fieldnames=header)
    writer.writeheader()
    for url in Url_all:
        driver.get(url)
        page_source=BeautifulSoup(driver.page_source,"html.parser")
        display=[]
        div1=page_source.find('div',class_='parameter')
        try :
            name=page_source.find('h1').get_text()
        except AttributeError:
            name='null'
            continue
        for a in div1.find('div',class_='liright'):
            if a.get_text()!='\n':
                display.append(a.get_text())
        display2= ",".join(display)#Thông tin màn hình
        path_li=div1.find_all('li')
        #print(type(hdh_path))
        #vì trong các thành phần được trích xuất từ bs4 có dạng mảng nên ta có thể trích xuất các phần tử
        # ở đây ta trích xuất phần tử từ trên xuống theo các phần tử có được trích xuất từ web 
        hdh=path_li[1].find('span').get_text() #Lấy thông tin hệ điều hành
        camera_sau=path_li[2].find('span').get_text() #Lấy thông tin camera sau
        camera_truoc=path_li[3].find('span').get_text() #Lấy thông tin camera trước
        chip=path_li[4].find('span').get_text() #Lấy thông tin chip
        ram=path_li[5].find('span').get_text() #Lấy thông tin ram
        rom=path_li[6].find('span').get_text() #Lấy thông tin rom
        try:
            sim=path_li[7].find('span').get_text() #Lấy thông tin sim
        except AttributeError:
            continue
        except IndexError:
            continue
        gia=page_source.find('p',class_='box-price-present').get_text() #Lấy thông tin giá
        battery=''
        charge=''
        
        
        
        # try:
        #     tmp=page_source.find('p',class_="detail-rate-total").get_text() #Lấy thông tin số lượng
        #     tmp=int(tmp.split(' ')[0])
        #     sluong=tmp
        # except AttributeError:
        #     sluong=0
        #     writer.writerow({ header[0]:name,header[1]:display2,header[2]:hdh,header[3]:camera_sau,header[4]:camera_truoc,header[5]:chip,header[6]:ram,header[7]:rom,header[8]:sim,header[9]:battery,header[10]:charge,header[11]:gia,header[12]:danhgia,header[13]:sluong})


            # continue
        i=0
        for tmp in  path_li[8].find('div'):
            if i==0 and tmp.get_text()!='\n':
                battery=battery+tmp.get_text()
                i=i+1
            else:
                charge=charge+tmp.get_text().strip()
        try :
            danhgia=page_source.find('p',class_='point').get_text() #Lấy thông tin đánh giá
            tmp=page_source.find('p',class_="detail-rate-total").get_text() #Lấy thông tin số lượng
            tmp=int(tmp.split(' ')[0])
            sluong=tmp
            writer.writerow({ header[0]:name,header[1]:display2,header[2]:hdh,header[3]:camera_sau,header[4]:camera_truoc,header[5]:chip,header[6]:ram,header[7]:rom,header[8]:sim,header[9]:battery,header[10]:charge,header[11]:gia,header[12]:danhgia,header[13]:sluong})
            print(name,display2,hdh,camera_sau,camera_truoc,chip,ram,rom,sim,battery,charge,gia,danhgia,sluong)
        except AttributeError:
                danhgia='NULL'
                sluong=0
                writer.writerow({ header[0]:name,header[1]:display2,header[2]:hdh,header[3]:camera_sau,header[4]:camera_truoc,header[5]:chip,header[6]:ram,header[7]:rom,header[8]:sim,header[9]:battery,header[10]:charge,header[11]:gia,header[12]:danhgia,header[13]:sluong})
                print(name,display2,hdh,camera_sau,camera_truoc,chip,ram,rom,sim,battery,charge,gia,danhgia,sluong)

                continue
        print(name,display2,hdh,camera_sau,camera_truoc,chip,ram,rom,sim,battery,charge,gia,danhgia,sluong)
        writer.writerow({ header[0]:name,header[1]:display2,header[2]:hdh,header[3]:camera_sau,header[4]:camera_truoc,header[5]:chip,header[6]:ram,header[7]:rom,header[8]:sim,header[9]:battery,header[10]:charge,header[11]:gia,header[12]:danhgia,header[13]:sluong})
            
    

Điện thoại Samsung Galaxy S22 Ultra 5G 128GB Dynamic AMOLED 2X,6.8",Quad HD+ (2K+) Android 12 Chính 108 MP & Phụ 12 MP, 10 MP, 10 MP 40 MP Snapdragon 8 Gen 1 8 nhân 8 GB 128 GB 2 Nano SIM hoặc 1 Nano SIM + 1 eSIM 5000 mAh 45 W 30.990.000₫  3.3 12
Điện thoại Samsung Galaxy S22 Ultra 5G 128GB Dynamic AMOLED 2X,6.8",Quad HD+ (2K+) Android 12 Chính 108 MP & Phụ 12 MP, 10 MP, 10 MP 40 MP Snapdragon 8 Gen 1 8 nhân 8 GB 128 GB 2 Nano SIM hoặc 1 Nano SIM + 1 eSIM 5000 mAh 45 W 30.990.000₫  3.3 12
Điện thoại iPhone 11 64GB IPS LCD,6.1",Liquid Retina iOS 15 2 camera 12 MP 12 MP Apple A13 Bionic 4 GB 64 GB 1 Nano SIM & 1 eSIM 3110 mAh 18 W 16.290.000₫  4.0 808
Điện thoại iPhone 11 64GB IPS LCD,6.1",Liquid Retina iOS 15 2 camera 12 MP 12 MP Apple A13 Bionic 4 GB 64 GB 1 Nano SIM & 1 eSIM 3110 mAh 18 W 16.290.000₫  4.0 808
Điện thoại OPPO Reno7 Z 5G AMOLED,6.43",Full HD+ Android 11 Chính 64 MP & Phụ 2 MP, 2 MP 16 MP Snapdragon 695 5G 8 nhân 8 GB 128 GB 2 Nano SIM (SIM 2 chung khe thẻ nhớ) 4500 mAh 

In [39]:
df=pd.read_csv('Du_lieu_dien_thoai2.csv')
df

,Name,Display,HDH,Camera sau,Camera trước,Chip,Ram,Rom,Sim,Battery,Charge,Gia,Danh gia,So Luong
0,Điện thoại Samsung Galaxy S22 Ultra 5G 128GB,"Dynamic AMOLED 2X,6.8"",Quad HD+ (2K+)",Android 12,"Chính 108 MP & Phụ 12 MP, 10 MP, 10 MP",40 MP,Snapdragon 8 Gen 1 8 nhân,8 GB,128 GB,2 Nano SIM hoặc 1 Nano SIM + 1 eSIM,5000 mAh,45 W,30.990.000₫,3.3,12
1,Điện thoại Samsung Galaxy S22 Ultra 5G 128GB,"Dynamic AMOLED 2X,6.8"",Quad HD+ (2K+)",Android 12,"Chính 108 MP & Phụ 12 MP, 10 MP, 10 MP",40 MP,Snapdragon 8 Gen 1 8 nhân,8 GB,128 GB,2 Nano SIM hoặc 1 Nano SIM + 1 eSIM,5000 mAh,45 W,30.990.000₫,3.3,12
2,Điện thoại iPhone 11 64GB,"IPS LCD,6.1"",Liquid Retina",iOS 15,2 camera 12 MP,12 MP,Apple A13 Bionic,4 GB,64 GB,1 Nano SIM & 1 eSIM,3110 mAh,18 W,16.290.000₫,4.0,808
3,Điện thoại iPhone 11 64GB,"IPS LCD,6.1"",Liquid Retina",iOS 15,2 camera 12 MP,12 MP,Apple A13 Bionic,4 GB,64 GB,1 Nano SIM & 1 eSIM,3110 mAh,18 W,16.290.000₫,4.0,808
4,Điện thoại OPPO Reno7 Z 5G,"AMOLED,6.43"",Full HD+",Android 11,"Chính 64 MP & Phụ 2 MP, 2 MP",16 MP,Snapdragon 695 5G 8 nhân,8 GB,128 GB,2 Nano SIM (SIM 2 chung khe thẻ nhớ),4500 mAh,33 W,10.490.000₫,3.8,69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Điện thoại Realme C21-Y 3GB,"IPS LCD,6.5"",HD+",Android 11,"Chính 13 MP & Phụ 2 MP, 2 MP",5 MP,Spreadtrum T610 8 nhân,3 GB,32 GB,2 Nano SIM,5000 mAh,10 W,3.690.000₫,3.7,15
140,Điện thoại Nokia C30,"IPS LCD,6.82"",HD+",Android 11 (Go Edition),Chính 13 MP & Phụ 2 MP,5 MP,Spreadtrum SC9863A,3 GB,32 GB,2 Nano SIM,6000 mAh,10 W,3.450.000₫,3.6,41
141,Điện thoại Nokia C30,"IPS LCD,6.82"",HD+",Android 11 (Go Edition),Chính 13 MP & Phụ 2 MP,5 MP,Spreadtrum SC9863A,3 GB,32 GB,2 Nano SIM,6000 mAh,10 W,3.450.000₫,3.6,41
142,Điện thoại Samsung Galaxy A03s,"PLS LCD,6.5"",HD+",Android 11,"Chính 13 MP & Phụ 2 MP, 2 MP",5 MP,MediaTek MT6765,4 GB,64 GB,2 Nano SIM,5000 mAh,7.75 W,3.690.000₫,3.7,178
